In [80]:
using Pkg
using LightGraphs
using IterTools
using DataStructures
using Base
using Combinatorics
#use Pkg.add("<<name>>") if you don't have a particular package

In [81]:
function valid_omino(omino,cont="rc")
    if 2 in omino
        return false
    end
    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = zeros(Int8, n1,n2)
    _f = false
    g = Grid([n1,n2])


    
    if cont == "qc" 
        for i=1:n1-1
           for j=1:n2-1

               add_edge!(g,((i-1)*n2+j),i*n2+j+1) 

            end


        end
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)

            end
        end
    end
    
    for i=n1:-1:1
        for j=n2:-1:1
           if omino[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end
           
        end   
    end
    
    return length(connected_components(g)) == 1
            
    
end
    


function make_omino(inds,dim,cont="rc")
    
    p = zeros(Int8,dim[1],dim[2])
    for loc in inds
        p[loc[1],loc[2]]=1
    end
    if valid_omino(p,cont)
        return p
    else
        return zeros(Int8,dim[1],dim[2])
    end

end









function make_omino_set(cells,grid,cont="rc")
    
   pair_idx = []
    for i=1:grid[1]
        for j=1:grid[2]
            push!(pair_idx, (i,j))
        end
    end
    ominos = []
    for t in subsets(pair_idx, Val{cells}())
        p = make_omino(t,grid,cont)
        
        if sum(p)!=0
            push!(ominos,p)
        end
    end
    
    
    return ominos
        
    
    
    
    
end


make_omino_set (generic function with 2 methods)

In [82]:
make_omino_set_qc(1,[4,4])

16-element Array{Any,1}:
 Int8[1 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 1 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 1 0; 0 0 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 1; 0 0 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 1 0 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 1 0 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 1 0; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 1; 0 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 1 0 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 1 0 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 1 0; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 0 1; 0 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 0 0; 1 0 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 0 0; 0 1 0 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 1 0]
 Int8[0 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 1]

In [83]:
function biggen(omlist,grid,bad,cont="rc")
    new_oms = Set()
    for o in omlist
        
        for i = 1:grid[1]
            for j = 1:grid[2]
                z = zeros(Int8,grid[1],grid[2])
                z[i,j]+=1
                if o[i,j]==0 && valid_omino(z+o,cont)
                    push!(new_oms,z+o)
 
                    
                end
            end
        end
    
        
        
        
        
        
    end
       
    return collect(new_oms)
end

biggen (generic function with 2 methods)

In [84]:
function check_holes(omino, bad_holes,cont="rc")

    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = ones(Int8, n1,n2)-omino
    _f = false
    g = Grid([n1,n2])
    
    
    if cont == "qc"
        for i=1:n1-1
           for j=1:n2-1

               add_edge!(g,((i-1)*n2+j),i*n2+j+1  ) 

            end


        end
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)

            end
        end        

    end
    
    
    
    
    
    for i=n1:-1:1
        for j=n2:-1:1
           if check[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end
            
            
            
            
            
        end   
    end
    
    for c in connected_components(g)
        if length(c) in bad_holes
            return false
        end
    end
    return true
            
    
    
end


check_holes (generic function with 2 methods)

In [85]:
function first_zero(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==0
                return (i,j)
            end
        end
    end
    return nothing
    
end
function first_one(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==1
                return (i,j)
            end
        end
    end
    return nothing
    
end

first_one (generic function with 1 method)

In [86]:
function build_conflicts(omlist, bad_holes,num_parts,cont="rc")
    
    
    firstdict = DefaultDict(Set)
    
    for o in 1:length(omlist)
        k = first_one(omlist[o])
        push!(firstdict[k],o)
    end
    

    conflict = DefaultDict(Set)
    print(length(omlist),"  ")
    for os in subsets(1:length(omlist),Val{2}())
        i = os[1]
        j = os[2]
        if i%500 == 0 && j==i+1
            print(i,"  ")
        end
        _c = true
            for p in zip(omlist[i],omlist[j])
                if _c && p[1]*p[2] ==1
                    _c = false
                end
            end
            if _c && check_holes(omlist[i]+omlist[j],bad_holes,cont)
                push!(conflict[i],j)
                push!(conflict[j],i)
            end
        
    end
    return firstdict,conflict
end





build_conflicts (generic function with 2 methods)

In [87]:
function get_next(fd, curr_inds, omlist)
    
   return [ d for d in fd[first_zero( sum([omlist[j] for j in curr_inds    ])     )] if verify(curr_inds + d)   ] 
    
end
            

get_next (generic function with 1 method)

In [88]:
function enumerator(grid, om_sizes,num_parts, cont="rc", io=false)
    count = 0
    oms = []
    bad_hole_sizes = [ [i for i = 1:minimum(om_sizes)-1] ;[i for i = maximum(om_sizes)+1:2*minimum(om_sizes)-1]        ]
    
    tmp_oms = make_omino_set(1,grid,cont)
    if io
        os = om_sizes
        gr = grid
        np = num_parts
        outfile = open(replace("enumerations/enum_$(gr)_$(os)_$(np)_$(cont).txt"," "=>""),"w")
    end
    
    
    if 1 in om_sizes
        oms = [oms;tmp_oms]
    end
    
    for sz = 2:maximum(om_sizes)
        tmp_oms = biggen(tmp_oms,grid,bad_hole_sizes,cont)
        if sz in om_sizes
            oms = [oms;tmp_oms]
        end
        #print("MADE SZ ",sz ,' ',length(tmp_oms),"\n")


    end
    
    
    oms = [ o for o in oms if check_holes(o,bad_hole_sizes,cont)]

    #print("MADE OMINOS\n")
    
    
    
    fd,cf = build_conflicts(oms,bad_hole_sizes,num_parts,cont)
    #print("MADE DICTIONARIES\n")           
    
    function verify(plan)
        #print(plan,'\n')
        
        for p in subsets(plan,Val{2}())
            if !(p[2] in cf[p[1]])
                return false
            end
        end
        return true
    end
           
                
                
    function recurs_part(plan,steps)
        #print("plan:",plan,"  sum: ",sum([oms[j] for j in plan]),'\n')
        if steps == 0
            if sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2] #&& sort!(collect([sum(oms[j]) for j in plan    ] )) == om_sizes
                count +=1
                if io
                    strrep = "$(sum([j.*oms[plan[j]] for j in 1:length(plan)]))\n"
                    strrep = replace(strrep," "=>",")
                    strrep = replace(strrep,"["=>"")
                    strrep = replace(strrep,";"=>"")
                    strrep = replace(strrep,"]"=>"")
                    write(outfile,  strrep)
                end
                return
            end
        elseif sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2]
        return
        end
        for q in fd[first_zero( sum([oms[j] for j in plan     ]))]
            if verify([plan;[q]])
                recurs_part([plan;[q]],steps-1)
            end
        end
    end
    
    #print("NEED TO DO ",length(fd[(1,1)]), " TOP-LEVELS\n")
    for p in fd[(1,1)]
        #print(" ",p," ")
        recurs_part([p],num_parts-1)
    end
    
    
   print('\n')
    if io
        close(outfile)
    end
   return count 
    
end

enumerator (generic function with 3 methods)

In [94]:
@time enumerator([3,5],[3],5,"rc",false)

42  
  0.014581 seconds (87.89 k allocations: 6.870 MiB, 30.17% gc time)


62

In [ ]:
@time enumerator([2,2],[2],2,"rc",true)
@time enumerator([2,2],[2],2,"qc",true)
@time enumerator([2,2],[1,2,3],2,"rc",true)
@time enumerator([2,2],[1,2,3],2,"qc",true)
@time enumerator([2,2],[1,2,3,4],2,"rc",true)
@time enumerator([2,2],[1,2,3,4],2,"qc",true)
@time enumerator([2,2],[1,2,3,4,5],2,"rc",true)
@time enumerator([2,2],[1,2,3,4,5],2,"qc",true)
@time enumerator([2,2],[1,2,3,4,5,6,7,8],2,"rc",true)
@time enumerator([2,2],[1,2,3,4,5,6,7,8],2,"qc",true)



@time enumerator([3,3],[3],3,"rc",true)
@time enumerator([3,3],[3],3,"qc",true)
@time enumerator([3,3],[2,3,4],3,"rc",true)
@time enumerator([3,3],[2,3,4],3,"qc",true)
@time enumerator([3,3],[1,2,3,4,5],3,"rc",true)
@time enumerator([3,3],[1,2,3,4,5],3,"qc",true)
@time enumerator([3,3],[1,2,3,4,5,6],3,"rc",true)
@time enumerator([3,3],[1,2,3,4,5,6],3,"qc",true)
@time enumerator([3,3],[1,2,3,4,5,6,7,8,9],3,"rc",true)
@time enumerator([3,3],[1,2,3,4,5,6,7,8,9],3,"qc",true)



@time enumerator([4,4],[4],4,"rc",true)
@time enumerator([4,4],[4],4,"qc",true)
@time enumerator([4,4],[3,4,5],4,"rc",true)
@time enumerator([4,4],[3,4,5],4,"qc",true)
@time enumerator([4,4],[2,3,4,5,6],4,"rc",true)
@time enumerator([4,4],[2,3,4,5,6],4,"qc",true)
@time enumerator([4,4],[1,2,3,4,5,6,7],4,"rc",true)
@time enumerator([4,4],[1,2,3,4,5,6,7],4,"qc",true)
@time enumerator([4,4],[i for i=1:16],4,"rc",true)
@time enumerator([4,4],[i for i=1:16],4,"qc",true)


In [ ]:
@time enumerator([4,4],[2],8,"rc",true)
@time enumerator([4,4],[2],8,"qc",true)
@time enumerator([4,4],[1,2,3],8,"rc",true)
@time enumerator([4,4],[1,2,3],8,"qc",true)
@time enumerator([4,4],[1,2,3,4],8,"rc",true)
@time enumerator([4,4],[1,2,3,4],8,"qc",true)
@time enumerator([4,4],[1,2,3,4,5],8,"rc",true)
@time enumerator([4,4],[1,2,3,4,5],8,"qc",true)
@time enumerator([4,4],[i for i=1:16],8,"rc",true)
@time enumerator([4,4],[i for i=1:16],8,"qc",true)



@time enumerator([4,4],[8],2,"rc",true)
@time enumerator([4,4],[8],2,"qc",true)
@time enumerator([4,4],[7,8,9],2,"rc",true)
@time enumerator([4,4],[7,8,9],2,"qc",true)
@time enumerator([4,4],[6,7,8,9,10],2,"rc",true)
@time enumerator([4,4],[6,7,8,9,10],2,"qc",true)
@time enumerator([4,4],[5,6,7,8,9,10,11],2,"rc",true)
@time enumerator([4,4],[5,6,7,8,9,10,11],2,"qc",true)
@time enumerator([4,4],[i for i=1:16],2,"rc",true)


In [ ]:
#=

@time enumerator([5,5],[5],5,"rc",true)
@time enumerator([5,5],[4,5,6],5,"rc",true)
@time enumerator([5,5],[3,4,5,6,7],5,"rc",true)
@time enumerator([5,5],[2,3,4,5,6,7,8],5,"rc",true)


@time enumerator([6,6],[6],6,"rc",true)
@time enumerator([6,6],[2],18,"rc",true)
@time enumerator([6,6],[3],12,"rc",true)
@time enumerator([6,6],[4],9,"rc",true)
@time enumerator([6,6],[9],4,"rc",true)
@time enumerator([6,6],[12],3,"rc",true)
@time enumerator([6,6],[2],18,"qc",true)





=#





#@time enumerator([5,5],[5],5,"qc",true)
@time enumerator([5,5],[4,5,6],5,"qc",true)
#@time enumerator([5,5],[3,4,5,6,7],5,"qc",true)
#@time enumerator([5,5],[2,3,4,5,6,7,8],5,"qc",true)



In [44]:
@time enumerator([3,3],[3],3,"rc",false)

ONES: 9
11  
  0.013547 seconds (13.42 k allocations: 1.077 MiB)


4

In [ ]:
@time enumerator_qc([5,5],[4,5,6],5)

UndefVarError: UndefVarError: sub2ind not defined

In [ ]:
enumerator()